In [ ]:
import numpy as np
import math
import cv2

def harris_corner(srcImage):
    
    grayImage = cv2.cvtColor(srcImage, cv2.COLOR_BGR2GRAY)
    color_Image = srcImage.copy()

    #derivative calculation
    #I_x, I_y = np.gradient(grayImage)
    
    I_x = cv2.Sobel(grayImage,cv2.CV_64F,1,0,ksize=3)
    I_y = cv2.Sobel(grayImage,cv2.CV_64F,0,1,ksize=3)

    Ixx = cv2.GaussianBlur(I_x**2,(3,3), 1)
    Ixy = cv2.GaussianBlur(I_x*I_y,(3,3), 1)
    Iyy = cv2.GaussianBlur(I_y**2,(3,3), 1)

    rows, columns = grayImage.shape
    cornerArray = np.zeros((rows,columns))
    windowSize = 3
    offset = int(windowSize/2)

    #find corners
    epsilon = 10**(-9)
    alpha = 0.04
    for y in range(offset, rows-offset):
        for x in range(offset, columns-offset):
            grad_Ixx = Ixx[y-offset:y+offset+1, x-offset:x+offset+1]
            grad_Ixy = Ixy[y-offset:y+offset+1, x-offset:x+offset+1]
            grad_Iyy = Iyy[y-offset:y+offset+1, x-offset:x+offset+1]

            S_Ixx = grad_Ixx.sum()
            S_Ixy = grad_Ixy.sum()
            S_Iyy = grad_Iyy.sum()

            determinantH = (S_Ixx*S_Iyy) - (S_Ixy*S_Ixy)
            traceH = S_Ixx + S_Iyy
            #cornerStrength = determinantH/(traceH+epsilon)
            cornerStrength = determinantH - alpha*(traceH**2)

            #print(x,y)
            cornerArray[y,x] = cornerStrength

    maxStrength = np.amax(cornerArray)
    thresh = 0.6*maxStrength
    
    for y in range(0,rows):
        for x in range(0,columns):
            #remove edge points and points less than threshold
            if cornerArray[y,x]< thresh or y-8<0 or y+8>rows or x-8<0 or x+8>columns:
                cornerArray[y,x] = 0
                
    #non maximum suppression
    harrisPoints = non_maxima_suppression(cornerArray, 5)
    #adaptive non maximum suppression
    newpoints = adaptive_max_suppression(harrisPoints)
    
    keypoints =[]
    for point in newpoints:
        x = point[0]
        y = point[1]
        keypoints.append(cv2.KeyPoint(x,y, _size = 1))
        cv2.drawMarker(color_Image, (int(x), int(y)), (0,0,255), 
                               markerType=cv2.MARKER_CROSS, markerSize=20, thickness=1, line_type=cv2.LINE_AA)
    
    return color_Image, keypoints

def writeArray(arr, name):
    outfile = open(name, 'w')
    rows, col = arr.shape
    for y in range(0,rows):
        for x in range(0,col):
            outfile.write(str(arr[y][x]) + " ")
        outfile.write('\n')

def non_maxima_suppression(cornerArray, window):
    print("Starting non maximum suppression")
    harrisPoints = []
    radius = int(window/2)
    rows, columns = cornerArray.shape
    for y in range(0,rows):
        for x in range(0,columns):
            if cornerArray[y,x]>0:
                arr = neighbors(radius, y, x, cornerArray)
                if cornerArray[y,x]<np.amax(arr):
                    continue
                    #cornerArray[y,x] = 0
                else:
                    harrisPoints.append([x,y,cornerArray[y,x]])
    return harrisPoints

def adaptive_max_suppression(harrisPoints):
    newPoints = []
    for hpoint in harrisPoints:
        minRadius = -1
        for point in harrisPoints:
            if hpoint[0] == point[0] and hpoint[1] == point[1]:
                continue
            elif hpoint[2]<0.9*point[2]:
                dx = point[0] - hpoint[0]
                dy = point[1] - hpoint[1]
                dist = math.sqrt(dx**2 + dy**2)
                if minRadius == -1:
                    minRadius = dist
                elif dist<minRadius:
                    minRadius = dist
        if minRadius == -1:
            minRadius = 999999999
        newPoints.append([hpoint[0], hpoint[1], hpoint[2], minRadius])
    newPoints = sorted(newPoints, key = lambda x:x[3], reverse = True)
    return newPoints

def sift_desc(image, features):
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grayImage = cv2.GaussianBlur(grayImage,(0,0),1)
    gradients = np.array(np.gradient(grayImage))
    magnitudes, ang = cv2.cartToPolar(gradients[0], gradients[1])
    feature_desc = []
    for item in features:
        x, y = item.pt
        y = int(y)
        x = int(x)
        feature =[]
        grid = grayImage[y-8:y+8+1, x-8:x+8+1]
        grid_magnitude = magnitudes[y-8:y+8+1, x-8:x+8+1]
        grid_normMag = np.zeros((17,17))
        cv2.normalize(grid_magnitude, grid_normMag)
        grid_angle = ang[y-8:y+8+1, x-8:x+8+1]
        #get rotation invarience
        rotated_grid = rot_invariance(grid_angle)
        
        arr = [0, 4, 9, 13]
        for i in range(0, len(arr)):
            for j in range(0, len(arr)):
                grid4_angle = rotated_grid[arr[i]:arr[i]+4, arr[j]:arr[j]+4]
                grid4_mag = grid_normMag[arr[i]:arr[i]+4, arr[j]:arr[j]+4]
                hist_array = get_histogram(grid4_angle, grid4_mag)
                feature.append(hist_array)
        feature_vector = np.asarray(feature, dtype ='float')
        feature_desc.append(feature_vector)
    return feature_desc
 
    
def rot_invariance(grid_angle):
    key_angle = grid_angle[8,8]
    rows, cols = grid_angle.shape
    new_grid = np.zeros((rows, cols))
    for i in range(0,rows):
        for j in range(0,cols):
            new_grid[i,j] = grid_angle[i,j]-key_angle
    return new_grid

def get_histogram(angle, magnitudes):
    hist_array = np.zeros(8)
    rows, cols = angle.shape
    for y in range(0, rows):
        for x in range(0,cols):
            degree =math.degrees(angle[y,x])
            if degree<0:
                degree+=360
            pos = int(degree/45)
            hist_array[pos] += magnitudes[y,x]
    #Normalize
    hist_array = normalize(hist_array)
    #for i in range(0,8):
        #if hist_array[i] == 0.2:
            #hist_array = normalize(hist_array)
            #break
    return hist_array

def normalize(histogram):
    new_array = np.zeros(8)
    sq_arr = np.square(histogram)
    ss = math.sqrt(sum(sq_arr))
    for i in range(0,8):
        new_array[i] = histogram[i]/ss
        if new_array[i]>0.2:
            new_array[i] = 0.2
    return new_array
        

def neighbors(radius, rowNumber, columnNumber, arr):
    out =[]
    for j in range(columnNumber-1-radius, columnNumber+radius):
        row=[]
        for i in range(rowNumber-1-radius, rowNumber+radius):
            if  i >= 0 and i < len(arr) and j >= 0 and j < len(arr[0]):
                row.append(arr[i][j])
            else:
                row.append(0)
        out.append(row)
    a = np.asarray(out, dtype=np.float32) 
    return a

def get_featureMatch(f_desc1, f_desc2):
    matches = []
    for index1, image1_features in enumerate(f_desc1):
        vecA = image1_features
        dist1 = 10
        dist2 = 10
        best_index2 = -1
        
        for index2, image2_features in enumerate(f_desc2):
            vecB = image2_features
            ssd =((vecA-vecB)**2).sum()
            if ssd < dist1:
                dist2 = dist1
                dist1 = ssd
                best_index2 = index2
        
        if (dist1/dist2)<0.6 :
            print("match: ", index1, best_index2)
            matches.append(cv2.DMatch(index1, best_index2, dist1))
            
    matches = sorted(matches, key=lambda x:x.distance)
    return matches

def main():
    
    print("Reading images...")
    #image1 = cv2.imread("C:/Users/shbi/Desktop/Shreya/Assignment2/image_sets/yosemite/Yosemite1.jpg")
    #image2 = cv2.imread("C:/Users/shbi/Desktop/Shreya/Assignment2/image_sets/yosemite/Yosemite2.jpg")
    
    image1 = cv2.imread("C:/Users/shbi/Desktop/Shreya/Assignment2/image_sets/graf/img1.ppm")
    image2 = cv2.imread("C:/Users/shbi/Desktop/Shreya/Assignment2/image_sets/graf/img2.ppm")
    
    print("Finding keypoints...")
    feature_image1, features1 = harris_corner(image1)
    feature_image2, features2 = harris_corner(image2)
    
    cv2.imwrite("feature_image1.png", feature_image1)
    cv2.imwrite("feature_image2.png", feature_image2)
    
    print("Computing feature descriptors ...")
    f_desc1 = sift_desc(image1, features1)
    f_desc2 = sift_desc(image2, features2)
    
    print("Feature mapping ...")
    matches = get_featureMatch(f_desc1, f_desc2)
    output = cv2.drawMatches(image1, features1, image2, features2, matches[:25], None, flags=2)
    cv2.imwrite("output.png", output)

if __name__ == '__main__':
    main()
    


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\shbi\AppData\Local\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-61af9eba414b>", line 3, in <module>
    import cv2
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 670, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 583, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 1043, in create_module
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\shbi\AppData\Local\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_rend

KeyboardInterrupt: 